#### Pre-requisites
- Create a secret scope using CLI or UI
- Store secrets in the scope using CLI or UI

In [0]:
%python

# Save the secrets as text widgets, not recommended for production
dbutils.widgets.text("PG_PASS", dbutils.secrets.get(scope="dea_scope", key="pg_password"))
dbutils.widgets.text("USER_PASS", dbutils.secrets.get(scope="dea_scope", key="pg_user"))

In [0]:
DROP CATALOG IF EXISTS rds_pg;
DROP CONNECTION IF EXISTS rds_pg_conn;

In [0]:
CREATE CONNECTION rds_pg_conn TYPE postgresql
OPTIONS (
  host 'database-1.c7qyscmoupbs.ca-central-1.rds.amazonaws.com',
  port '5432',
  user :PG_USER,
  password :PG_PASS
)

In [0]:
CREATE FOREIGN CATALOG rds_pg
USING CONNECTION rds_pg_conn
OPTIONS (database 'postgres');

In [0]:
SELECT * FROM rds_pg.public.refunds LIMIT 10;

refund_id,payment_id,refund_timestamp,refund_amount,refund_reason
1,66,2025-01-10T11:30:00.000Z,85.75,Payment Error:Retailer
2,69,2025-01-03T12:40:15.000Z,120.50,Order Cancelled:Customer
3,72,2025-01-06T14:45:30.000Z,65.00,Product Returned:Customer
4,73,2025-01-07T16:10:45.000Z,210.99,Order Cancelled:Customer
5,75,2025-01-09T18:25:00.000Z,45.20,Payment Error:Retailer
6,80,2025-01-10T09:35:20.000Z,130.15,Order Cancelled:Customer
7,83,2025-01-12T11:20:40.000Z,150.00,Product Returned:Customer
8,85,2025-01-14T13:15:30.000Z,89.99,Order Cancelled:Customer
9,89,2025-01-15T15:00:00.000Z,78.50,Payment Error:Retailer
10,91,2025-01-17T16:45:15.000Z,250.75,Product Returned:Customer


In [0]:
CREATE OR REPLACE VIEW gizmobox.bronze.v_refunds AS
SELECT * FROM rds_pg.public.refunds;

In [0]:
SELECT * FROM gizmobox.bronze.v_refunds LIMIT 10;

refund_id,payment_id,refund_timestamp,refund_amount,refund_reason
1,66,2025-01-10T11:30:00.000Z,85.75,Payment Error:Retailer
2,69,2025-01-03T12:40:15.000Z,120.50,Order Cancelled:Customer
3,72,2025-01-06T14:45:30.000Z,65.00,Product Returned:Customer
4,73,2025-01-07T16:10:45.000Z,210.99,Order Cancelled:Customer
5,75,2025-01-09T18:25:00.000Z,45.20,Payment Error:Retailer
6,80,2025-01-10T09:35:20.000Z,130.15,Order Cancelled:Customer
7,83,2025-01-12T11:20:40.000Z,150.00,Product Returned:Customer
8,85,2025-01-14T13:15:30.000Z,89.99,Order Cancelled:Customer
9,89,2025-01-15T15:00:00.000Z,78.50,Payment Error:Retailer
10,91,2025-01-17T16:45:15.000Z,250.75,Product Returned:Customer


In [0]:
DESC EXTENDED gizmobox.bronze.v_refunds;

col_name,data_type,comment
refund_id,int,null
payment_id,int,null
refund_timestamp,timestamp,null
refund_amount,"decimal(10,2)",null
refund_reason,varchar(255),null
,,
# Detailed Table Information,,
Catalog,gizmobox,
Database,bronze,
Table,v_refunds,


In [0]:
CREATE OR REPLACE TABLE gizmobox.bronze.refunds
AS
SELECT * FROM rds_pg.public.refunds;

num_affected_rows,num_inserted_rows


In [0]:
SELECT * FROM gizmobox.bronze.refunds LIMIT 10;

refund_id,payment_id,refund_timestamp,refund_amount,refund_reason
1,66,2025-01-10T11:30:00.000Z,85.75,Payment Error:Retailer
2,69,2025-01-03T12:40:15.000Z,120.50,Order Cancelled:Customer
3,72,2025-01-06T14:45:30.000Z,65.00,Product Returned:Customer
4,73,2025-01-07T16:10:45.000Z,210.99,Order Cancelled:Customer
5,75,2025-01-09T18:25:00.000Z,45.20,Payment Error:Retailer
6,80,2025-01-10T09:35:20.000Z,130.15,Order Cancelled:Customer
7,83,2025-01-12T11:20:40.000Z,150.00,Product Returned:Customer
8,85,2025-01-14T13:15:30.000Z,89.99,Order Cancelled:Customer
9,89,2025-01-15T15:00:00.000Z,78.50,Payment Error:Retailer
10,91,2025-01-17T16:45:15.000Z,250.75,Product Returned:Customer


In [0]:
DESC EXTENDED gizmobox.bronze.refunds;

col_name,data_type,comment
refund_id,int,null
payment_id,int,null
refund_timestamp,timestamp,null
refund_amount,"decimal(10,2)",null
refund_reason,varchar(255),null
,,
# Delta Statistics Columns,,
Column Names,"refund_id, refund_timestamp, refund_amount, refund_reason, payment_id",
Column Selection Method,first-32,
,,


In [0]:
-- [NOT_SUPPORTED_WITH_SERVERLESS] REFRESH TABLE is not supported on serverless compute. SQLSTATE: 0A000
-- REFRESH TABLE gizmobox.bronze.refunds;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8238969295448539>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'REFRESH TABLE gizmobox.bronze.refunds;\n')

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2543, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2541 with self.builtin_trap:
   2542     args = (magic_arg_s, cell)
-> 2543     result = fn(*args, **kwargs)
   2545 # The code below prevents the output from being displayed
   2546 # when using magics with decorator @output_can_be_silenced
   2547 # when the last Python token in the expression is a ';'.
   2548 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:130, in SqlMagic.sql(self, line, cell)
    124 except BaseException as e:
    125     self.driver_activity_logger

In [0]:
INSERT OVERWRITE gizmobox.bronze.refunds
SELECT * FROM rds_pg.public.refunds;

num_affected_rows,num_inserted_rows
10,10


In [0]:
SELECT * FROM gizmobox.bronze.refunds LIMIT 10;

refund_id,payment_id,refund_timestamp,refund_amount,refund_reason
1,66,2025-01-10T11:30:00.000Z,85.75,Payment Error:Retailer
2,69,2025-01-03T12:40:15.000Z,120.50,Order Cancelled:Customer
3,72,2025-01-06T14:45:30.000Z,65.00,Product Returned:Customer
4,73,2025-01-07T16:10:45.000Z,210.99,Order Cancelled:Customer
5,75,2025-01-09T18:25:00.000Z,45.20,Payment Error:Retailer
6,80,2025-01-10T09:35:20.000Z,130.15,Order Cancelled:Customer
7,83,2025-01-12T11:20:40.000Z,150.00,Product Returned:Customer
8,85,2025-01-14T13:15:30.000Z,89.99,Order Cancelled:Customer
9,89,2025-01-15T15:00:00.000Z,78.50,Payment Error:Retailer
10,91,2025-01-17T16:45:15.000Z,250.75,Product Returned:Customer
